In [1]:
import csv
import os
import re
from itertools import islice

### csv to yearcsv

In [4]:
def getstuff2(year):
    written = 0
    with open('dbs_orig\lenta-ru-news.csv', mode='r', encoding='utf-8') as csvfile:
        datareader = csv.reader(csvfile)
        with open(f'{year}.csv', mode='a+', encoding='utf-8') as output:
            #output.write('url,title,text,topic,tags,date')
            for num, row in enumerate(datareader):
                if row[5].startswith(f'{year}'):
                    row[2] = row[2].replace('\r\n', ' ').replace('\n', ' ').replace('\r', ' ')
                    output.write('\n' + ','.join(row))  # or tsv?
                    written = 1
                elif not row[0].startswith('http') and written == 1:
                    output.write(''.join(row))
                elif row[5].startswith(f'{year+1}'):
                    written = 0
                    print('this is the end')
                    break

In [3]:
getstuff2(2013)

this is the end


### с ДБ

In [2]:
import sqlite3

In [3]:
vz = sqlite3.connect('vzglyad.db')
cur = vz.cursor()

In [17]:
year = 2017
db2csv(year)

In [16]:
# USE THIS year, month, day, news

def db2csv(year):
    cur.execute(f'SELECT year,month,day,text FROM news WHERE year = {year}')
    with open (f'vz_{year}.txt', 'w+', encoding='utf-8') as f:
        for row in cur.fetchall():
            row = list(row)
            if not year == 2017:
                row[3] = fixcapitals(row[3]).replace('\n', ' ').strip(' ')
            else:
                row[3] = row[3].replace('\n', ' ').replace('\xa0', ' ').replace('\r', ' ').replace('\t', ' ').strip(' ')
            if row[1]<10:
                row[1] = '0'+str(row[1])
            if row[2]<10:
                row[2]= '0'+str(row[2])
            f.write('\t'.join([str(it) for it in row]) + '\n')

In [5]:
def fixcapitals(text):
    abbrlist = ['КоАП', 'СПб', 'СПбГУ', 'УЭБиПК', 'ГУЭБиПК', 'ИрГТУ'
               'МегаФон', 'МегаФоном', 'МегаФону', 'ШАСизмом', 'ШАСизм',
               'ШАСизма', 'ШАСизму', 'АэроСвита', 'ЮТэйр', 'АКИпресс', 'ГАЗель', 'ГАЗели',
               'ЕврАз', 'ЛиАЗ', 'ЭкзоМарс', 'КамАЗ', 'МиГ', 'МиГи', 'МиГа', 'КоммерсантЪ',
               'ВКонтакте', 'КнААПО', 'СвДП', 'УрГЭУ', 'ЕвроПРО', 'КОИБами',
               'ЦНИИмаш', 'ЦНИИмаше', 'ЦНИИмаша', 'ЦНИИмашу', 'РосРАО',
               'НТВшники', 'АвтоВАЗ', 'ЦНИИмаш', 'ДнепроГЭС', 'ЧОПовцев', 'ЧОПовцев', 'ЧОПовцы', 'ЧОПовца']
    text = text.replace('\xa0', ' ').replace('\r', ' ')
    text = text.replace('СШАПо', 'США По')
    text = text.replace('.', '. ')
    text = re.sub('[^\w0-9’ -]', ' ', text)
    tokens = text.split(' ')
    for num, token in enumerate(tokens):
        if token in abbrlist:
            pass
        else:
            countcapital = 0
            
            for char in token:
                if char.isupper():
                    countcapital +=1
        
            if re.sub('([A-z]+)', ' \g<1> ', token) != token:
                if re.search('[А-ЯЁа-яё]', token):
                    tokens[num] = re.sub('([A-z]+)', ' \g<1> ', token)  
            elif all(char.isupper() for char in token):
                pass
            elif not token.replace('-', '').isalpha():
                pass
            elif countcapital>=1 and token[0].islower():
                if re.findall('([а-яё]+)([А-ЯЁ]+[а-яё]+)', token):
                    split = re.findall('([а-яё]+)([А-ЯЁ]+[а-яё]+)', token)[0]
                    tokens[num] = ' '.join(split)
                elif re.findall('([а-яё]+)([А-ЯЁ]+)', token):
                    split = re.findall('([а-яё]+)([А-ЯЁ]+)', token)[0]
                    tokens[num] = ' '.join(split)
                else:
                    pass
                
            elif countcapital>1:
                t1 = tokens[num]
                if re.findall('([А-ЯЁ][а-яё]+)([А-ЯЁ][а-яё]+)', token):
                    split = re.findall('([А-ЯЁ][а-яё]+)([А-ЯЁ][а-яё]+)', token)[0]
                    tokens[num] = ' '.join(split)
                elif re.findall('([А-ЯЁ]+[а-яё][а-яё]?[А-ЯЁ]+)', token):
                    test = re.findall('([А-ЯЁ]+[а-яё][а-яё]?[А-ЯЁ]+)', token)
                    if token in test:
                        pass
                   # else:
                    #    print(token)
                elif re.findall('([А-ЯЁ][а-яё]+)([А-ЯЁ]+)', token):
                    split = re.findall('([А-ЯЁ][а-яё]+)([А-ЯЁ]+)', token)[0]
                    tokens[num] = ' '.join(split)
    
                elif re.findall('([А-ЯЁ]+)([А-ЯЁ]+[а-яё][а-яё][а-яё]+)', token):
                    split = re.findall('([А-ЯЁ]+)([А-ЯЁ]+[а-яё][а-яё][а-яё]+)', token)[0]
                    tokens[num] = ' '.join(split)
                    #if t1!=tokens[num]:
                     #   print(tokens[num])
                else:  # change nothing
                    pass   
                
            else:
                pass
  
    text = ' '.join(tokens)
    text = re.sub(' +', ' ', text)

    return text

### lemmatize news

In [14]:
# forlemmatization patterns
poempattern = re.compile('author>(.+?)<review num', re.MULTILINE|re.DOTALL)
tspattern = re.compile('[\t|\s]+')
linkspattern = re.compile('<a href.+?</a>')
# cleanlemmatized
notworddigittabhyph = re.compile('[^\w0-9-\t]')
underscore = re.compile('_')
multiplehyphens = re.compile('-+?')
righthyphen = re.compile(' -')
lefthyphen = re.compile('- ')
hyphen = re.compile(' - ')
multiplespace = re.compile(' +')

In [23]:
def full(inFile, outFile):
    with open(inFile, encoding='utf-8') as f:
        with open(outFile, 'w+', encoding='utf-8') as t:
            for news in islice(f, 1, None):
                items = news.split(',')
                t.write(items[0] + '$' + ','.join(items[2:-3]) + '\n')
                
def cleanlemmatized(lemmatized, outputfname):
    with open(lemmatized, 'r', encoding='utf-8') as f:
        text = f.read()
    with open (outputfname, 'w+', encoding='utf-8') as output:
        output.write('year\tmonth\tday\tnews')
        for line in text.splitlines():
            if re.search('}/(\d{4})/(\d{2})/(\d{2})', line):
                output.write('\n')
            try:
                link, news = line.split('$', 1)
                date = re.search('(\d{4})/(\d{2})/(\d{2})', link)
                year = date.group(1)
                month = date.group(2)
                day = date.group(3)
                clean = notworddigittabhyph.sub(' ', news)
                clean = underscore.sub('', clean)
                clean = multiplehyphens.sub('-', clean)
                clean = righthyphen.sub(' ', clean)
                clean = lefthyphen.sub(' ', clean)
                clean = multiplespace.sub(' ', clean)
                output.write('\t'.join([year, month, day, clean.lower().replace('\t', ' ')]))
            except (ValueError, AttributeError):
                words = re.findall('{([-\w0-9]+)?}', line)
                output.write(' '.join(words).lower())
                
def lentatotsv(newscsv, forlem, lemmatized, outputtsv):
    full(newscsv, forlem)
    os.system(f'mystem.exe {forlem} {lemmatized} -c -l -d')
    cleanlemmatized(lemmatized, outputtsv)

In [ ]:
# ЗАПУСКАТЬ ЭТО -> ЛЕНТА
lentatotsv('2018.csv', '2018forlem.txt', '201lemmatized.txt', 'full2017.tsv')

In [13]:
def poemclean(lemmatized, outputfname): #checkiflatest
    with open(lemmatized, 'r', encoding='utf-8') as f:
        text = f.read()
    with open (outputfname, 'w+', encoding='utf-8') as output:
        for line in text.splitlines():
            clean = notworddigittabhyph.sub(' ', line)
            clean = underscore.sub('', clean)
            clean = multiplehyphens.sub('-', clean)
            clean = righthyphen.sub(' ', clean)
            clean = lefthyphen.sub(' ', clean)
            clean = multiplespace.sub(' ', clean)
            output.write(clean.strip(' ').lower() + '\n')
            
def vztotsv(forlem, lemmatized, outputtsv):
    os.system(f'mystem.exe {forlem} {lemmatized} -c -l -d')
    poemclean(lemmatized, outputtsv)

In [18]:
# ЗАПУСКАТЬ ЭТО -> ВЗГЛЯД

years = [2017]
for year in years:
    vztotsv(f'vz_{year}.txt', f'vz_{year}_lemmatized.txt', f'{year}vz.tsv')